### Synchronize magnet temperature and machine data

In [ ]:
import numpy as np
import pandas as pd
import pymongo as py
from datetime import datetime, timedelta 

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as ex
import plotly.io as pio

In [ ]:
from readIn import readIn
from synchronize import synchronize

In [ ]:
# Start and end time of the database request
# The database collects the data in UTC time!
start_sec= datetime.strptime("2023-09-06T07:20:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")
end_sec= datetime.strptime("2023-09-07T07:30:00.000+0000", "%Y-%m-%dT%H:%M:%S.%f+0000")

# Paths were magnet temperature is stored and where the combined file should be saved to
magnetDataPath = "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\"
combinedFilePath = "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\combined\\synchronized_2023_09_08_Rotor_combined.csv"

# Timeshift between magnet and machine data (summertime: 1, wintertime: 2)
timeshift = 1

In [ ]:
# Read in the magnet temperatures as a dataframe
df_magnet = readIn(magnetDataPath)
df_magnet= df_magnet.rename(columns = {"Date/Time" : "date"})
df_magnet = df_magnet.drop(columns = "filename")

In [ ]:
df_magnet.head(5)

In [ ]:
#####DB Config machine data
host= "m53763edge"
port=27017
collection= "thermal_data"

## Connect to data base and get the events
client = py.MongoClient(host= host, port=port)
db = client.h4ai
event_list = db[collection].find({ "date" : { '$gte' : start_sec, '$lt' : end_sec} }).sort('date', 1)

In [ ]:
# Read in the machine data
df = pd.DataFrame()
for event in event_list:
    content = event["content"]
    for data in content:
        raw_data = data["raw_data"]
        keylist = raw_data.keys()
        #keylist = list(keylist)
        raw_data["date"] = data["date"]
        raw_data["given2model"] = data["given2model"]
        raw_data["prediction"] = data["prediction"]
        df_row = pd.DataFrame(raw_data, index= ["date"])
        df = pd.concat([df, df_row], axis=0, ignore_index=True)
df.reset_index(inplace=True)
#df.to_csv(machinedata)
print(df.head(5))     

In [ ]:
scatter_mode= 'lines'

#nrrows = len(df.columns)+1
nrrows = 1
fig= make_subplots(rows=nrrows, cols=1, shared_xaxes= True, print_grid= True, vertical_spacing=0.01)
col_names = df.columns.values
# i = 0 
# for column in df:
#     fig.add_trace(go.Scatter(x= df['date'], y = df[column], name= col_names[i], mode= scatter_mode), row= i+1, col= 1)
#     i += 1
# fig.add_trace(go.Scatter(x= df_magnet['date'], y = df_magnet["magnet_temperature"], name= "magnet_temperature", mode= scatter_mode), row= nrrows, col= 1)

fig.add_trace(go.Scatter(x= df['date'], y = df["T_MOTOR"], name= "T_Motor", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df['date'], y = df["T_LAGER"], name= "T_Lager", mode= scatter_mode), row= nrrows, col= 1)

fig.add_trace(go.Scatter(x= df_magnet['date'], y = df_magnet["No.1"], name= "No.1", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_magnet['date'], y = df_magnet["No.2"], name= "No.2", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_magnet['date'], y = df_magnet["No.3"], name= "No.3", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_magnet['date'], y = df_magnet["No.4"], name= "No.4", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_magnet['date'], y = df_magnet["No.5"], name= "No.5", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_magnet['date'], y = df_magnet["No.6"], name= "No.6", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_magnet['date'], y = df_magnet["No.7"], name= "No.7", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_magnet['date'], y = df_magnet["No.8"], name= "No.8", mode= scatter_mode), row= nrrows, col= 1)

fig.update_layout(height=500, width=1300, title_text="Magnettemperaturen und Maschinendaten vor der Synchronisation und Zeitverschiebung")
fig.show()

In [ ]:
df_combined = synchronize(df_machine = df, df_magnet = df_magnet, timeshift = timeshift)

In [ ]:
scatter_mode= 'lines'

nrrows = 1
fig= make_subplots(rows=nrrows, cols=1, shared_xaxes= True, print_grid= True, vertical_spacing=0.01, specs=[[{"secondary_y": True}]])
col_names = df.columns.values

fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["T_MOTOR"], name= "T_Motor", mode= scatter_mode), row= nrrows, col= 1, secondary_y = False)
fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["T_LAGER"], name= "T_Lager", mode= scatter_mode), row= nrrows, col= 1, secondary_y = False)
fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["prediction"], name= "prediction", mode= scatter_mode), row= nrrows, col= 1, secondary_y = False)
fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["Analytic_sol"], name= "Analytic_sol", mode= scatter_mode), row= nrrows, col= 1, secondary_y = False)

fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["No.1"], name= "Temp. No.1", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["No.2"], name= "Temp. No.2", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["No.3"], name= "Temp. No.3", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["No.4"], name= "Temp. No.4", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["No.5"], name= "Temp. No.5", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["No.6"], name= "Temp. No.6", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["No.7"], name= "Temp. No.7", mode= scatter_mode), row= nrrows, col= 1)
fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["No.8"], name= "Temp. No.8", mode= scatter_mode), row= nrrows, col= 1)

# fig.add_trace(go.Scatter(x= df_combined['date'], y = df_combined["magnet_temperature"], name= "magnet_temperature", mode= scatter_mode), row= nrrows, col= 1, secondary_y = True)
# fig.add_trace(go.Scatter(x= df_magnet['date'], y = df_magnet["magnet_temperature"], name= "old magnet", mode= scatter_mode), row= nrrows, col= 1, secondary_y = True)

fig.update_layout(height=500, width=1300, title_text="Magnettemperaturen und Maschinendaten nach der Synchronisation")
fig.show()

In [ ]:
df_combined.to_csv(combinedFilePath)

In [ ]:
scatter_mode= 'lines'

params = ['date', "DRZ5", "T_KLEMMUNG", "T_LAGER", "T_MOTOR", 'prediction', 'Analytic_sol',
          'No.1', 'No.2', 'No.3', 'No.4', 'No.5', 'No.6', 'No.7', 'No.8']

df_plot = df_combined[params]
nrrows = 5
fig= make_subplots(rows=nrrows, cols=1, shared_xaxes= True, print_grid= True, vertical_spacing=0.01) #, specs=[[{"secondary_y": True}]]
col_names = df_plot.columns.values

i = 0 
j = 0
for column in df_plot:
    if column in ["DRZ5"]:
        fig.add_trace(go.Scatter(x= df_plot['date'], y = df_plot[column], name= col_names[i+1], mode= scatter_mode), row= i+1, col= 1)
        fig.update_yaxes(title_text="DRZ [RPM]", row=i+1, col=1)
        i += 1
    if column in ["T_KLEMMUNG", "T_LAGER", "T_MOTOR"]:
        fig.add_trace(go.Scatter(x= df_plot['date'], y = df_plot[column], name= col_names[i+1], mode= scatter_mode), row= i+1, col= 1)
        fig.update_yaxes(title_text=(column + " [°C]"), row=i+1, col=1)
        i += 1
    if column in ['prediction', 'Analytic_sol', 'No.1', 'No.2', 'No.3', 'No.4', 'No.5', 'No.6', 'No.7', 'No.8']:
        fig.add_trace(go.Scatter(x= df_plot['date'], y = df_plot[column], name= col_names[i+j+1], mode= scatter_mode), row= i+1, col= 1)
        fig.update_yaxes(title_text="Rotor Temperatur [°C]", row=i+1, col=1)
        j +=1

fig.update_layout(height=800, width=1100, title_text="Magnettemperaturen und Maschinendaten nach der Synchronisation")
fig.show()